### Exemplo de utilização da API do Twitter e utilização do MongoDB
### Análise de polaridade

In [21]:
import tweepy #instalado utilizando conda install -c conda-forge tweepy
from textblob import TextBlob as tb #instalado utilizando conda install -c conda-forge textblob
import json
import numpy as np
import pymongo #instalado utilizando conda install -c anaconda pymongo

In [2]:
#Credenciais
app_name = "aula_igti"
api_key = "por chave aqui"
api_secret_key = "por chave aqui"
access_token = "por chave aqui"
access_token_secret = "por chave aqui"

In [3]:
#Autenticação na API utilizando as credenciais
auth = tweepy.OAuthHandler(api_key, api_secret_key)
auth.set_access_token(access_token, access_token_secret)
token = tweepy.API(auth)

#### Busca por palavra-chave

In [4]:
#Determinando as palavras-chaves a serem procuradas
keyword = ("covid OR covid-19 OR covid19 OR corona OR coronavirus")

In [5]:
#Realizando a busca dos tweets
tweets = token.search(q=keyword, count=10, result_type='mixed', tweet_mode='extended')

In [6]:
#Checando a quantidade de tweets coletados
print("Total de tweets coletados: %s" %(len(tweets)))

Total de tweets coletados: 10


In [9]:
#Imprimindo os tweets
for tweet in tweets:
    print("Usuário: %s" %{tweet.user.screen_name})
    print("Tweet: %s" %{tweet.full_text}) #tweet.text, caso o tweet_mode não seja extended

Usuário: {'petemuntean'}
Tweet: {'Breaking - UPS and FedEx trucks carrying the first U.S. shipment of coronavirus vaccine have left Pfizer’s facility near Kalamazoo, Michigan. https://t.co/Cf32ki9gCF'}
Usuário: {'joncoopertweets'}
Tweet: {'Who else is outraged that high-ranking White House officials, who spent months refusing to wear masks and long supported Trump’s claims that Covid-19 is a hoax, will be among the first to get vaccinations — BEFORE nearly all high-risk health care workers &amp; nursing home residents?'}
Usuário: {'bowertttt'}
Tweet: {'RT @lifeofthornpi: ระดมทุน 500 บาท 1 ล้านคน เพื่อพัฒนาวัคซีน covid คือแบบ เฮ้อ กุต้องบริจาคอีกกี่ครั้ง แล้วภาษีที่จ่ายเอาไปทำหอกอะไร'}
Usuário: {'raffo20'}
Tweet: {'RT @MatLabPer: #coronavirus #COVID19 #COVIDー19 #Covid_19 #COVID #vacuna #Vacunas #vacunacion #segundaola #PeruEstaEnNuestrasManos #perutequ…'}
Usuário: {'i_am_leticia'}
Tweet: {"RT @bodinholoiro: 'o impeachment tiraria o foco do covid'\ncreio que o covid não iria se aborrecer

#### Análise de polaridade

Para realizar a **análise de polaridade**, utilizaremos a função *sentiment.polarity*, do pacote *Textblob*.

- A função retornará um valor entre -1 e 1, onde quanto maior o valor retornado, menos "chateada" a pessoa está, ou seja, mais positivo é o *tweet*;
- A polaridade 0 indica neutralidade do *tweet*.

In [10]:
analysis = None #irá armazenar as polaridades
tweets_score = [] #irá armazenar os scores

In [11]:
for tweet in tweets:
    print("**", tweet.full_text)
    analysis = tb(tweet.full_text)
    polarity = analysis.sentiment.polarity
    tweets_score.append(polarity)

** Breaking - UPS and FedEx trucks carrying the first U.S. shipment of coronavirus vaccine have left Pfizer’s facility near Kalamazoo, Michigan. https://t.co/Cf32ki9gCF
** Who else is outraged that high-ranking White House officials, who spent months refusing to wear masks and long supported Trump’s claims that Covid-19 is a hoax, will be among the first to get vaccinations — BEFORE nearly all high-risk health care workers &amp; nursing home residents?
** RT @lifeofthornpi: ระดมทุน 500 บาท 1 ล้านคน เพื่อพัฒนาวัคซีน covid คือแบบ เฮ้อ กุต้องบริจาคอีกกี่ครั้ง แล้วภาษีที่จ่ายเอาไปทำหอกอะไร
** RT @MatLabPer: #coronavirus #COVID19 #COVIDー19 #Covid_19 #COVID #vacuna #Vacunas #vacunacion #segundaola #PeruEstaEnNuestrasManos #perutequ…
** RT @bodinholoiro: 'o impeachment tiraria o foco do covid'
creio que o covid não iria se aborrecer
** RT @ChilangoCom: #ÚLTIMAHORA este es el saldo hasta el momento de casos de #COVID19 en México:   

1,255,974 confirmados   
114,298 defunci…
** RT @Aeskulappir

In [12]:
print("Vetor de Polaridade: ", tweets_score)

Vetor de Polaridade:  [0.11666666666666665, -0.11666666666666668, 0.0, 0.0, 0.0, 0.0, 0.0, 0.25, 0.0, 0.0]


In [13]:
print("Média de Sentimento: " + str(np.mean(tweets_score)))

Média de Sentimento: 0.024999999999999998


#### Análise de polaridade para *tweets* que não foram escritos em inglês

In [15]:
polarities = []

for tweet in tweets:
    analysis = tb(tweet.full_text)
    if analysis.detect_language() != 'en':
        traducao = tb(str(analysis.translate(to='en')))
        print('Texto traduzido %s: ' %traducao)
        polarity = traducao.sentiment.polarity
    else:
        polarity = analysis.sentiment.polarity
        
    polarities.append(polarity)

Texto traduzido RT @lifeofthornpi: Raised 500 baht, 1 million people to develop a covid vaccine. And what is the tax paid to make the spear?: 
Texto traduzido RT @MatLabPer: #coronavirus # COVID19 #COVID ー 19 # Covid_19 #COVID #vacuna #Vacunas #vacunacion #segundaola #PeruEstaEnNuestrasManos # perutequ…: 
Texto traduzido RT @bodinholoiro: 'impeachment would take the focus off the covid'
I believe the covid would not be bored: 
Texto traduzido RT @ChilangoCom: # LAST HOUR this is the balance so far of # COVID19 cases in Mexico:

1,255,974 confirmed
114,298 deaths ...: 
Texto traduzido RT @Aeskulappirat: Some contemporaries have to explain it in a child-friendly manner.

#Covidioten 🤪 #Coronazis #Solidarity deniers # WearAMa ...: 
Texto traduzido @ Tahun2049 Corona wes meet vaccines, let's report ora eroh hahaha: 


In [18]:
print('Vetor de polaridade: ', polarities)

Vetor de polaridade:  [0.11666666666666665, -0.11666666666666668, 0.0, 0.0, -0.5, 0.16666666666666666, 0.0, 0.25, 0.2, 0.0]


In [19]:
print("Média de Sentimento: " + str(np.mean(polarities)))

Média de Sentimento: 0.011666666666666662


#### Armazenando os arquivos coletados

In [22]:
status = tweets[0]

#converte para string
json_str = json.dumps(status._json)

#deserialise string para um objeto python
parsed = json.loads(json_str)

In [23]:
type(tweets)

tweepy.models.SearchResults

In [24]:
type(json_str)

str

In [25]:
type(parsed)

dict

In [27]:
datasetPath = r"C:\Users\caiov\Documents\_Profissional\Data Science\bootcamp-igti---cientista-de-dados\Módulo 2\Exemplos Aulas"

with open(datasetPath+r'\tweets_keywords.json', 'a', encoding='utf-8') as filename:
    for tweet in tweets:
        status = tweet
        json_str = json.dumps(status._json)
        parsed = json.loads(json_str)
        json.dump(parsed, filename, ensure_ascii=False, sort_keys=True, indent=4, separators=(',', ':'))

#### Armazenar no MongoDB

In [28]:
#Conexão ao banco de dados
con = pymongo.MongoClient('localhost', 27017)

#Seleciona o banco de dados
db = con.twitterdb

In [29]:
#Seleciona a coleção para armazenar os tweets
collection = db.tweets_keywords

In [31]:
#Lista a quantidade de tweets existentes na coleção
db.tweets_keywords.count()

#O aviso nos diz que devemos usar count_documents(), visto que count() foi descontinuado

<ipython-input-31-d7fd65803842>:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  db.tweets_keywords.count()


0

In [32]:
#Lista a quantidade de tweets existentes na coleção
db.tweets_keywords.count_documents({})

0

In [34]:
i=0

for tweet in tweets:
    db.tweets_keywords.insert_one(tweet._json)
    i+=1

In [35]:
#Lista a quantidade de tweets inseridos
print('Quantidade de tweets inseridos: %s' %i)

Quantidade de tweets inseridos: 10


In [36]:
#Lista a quantidade de tweets existentes na coleção
db.tweets_keywords.count_documents({})

10